### As usual we setup Weaviate

In [7]:
%env GEMINI_API_KEY=
                   
import requests
import json

# Download the data
resp = requests.get('https://raw.githubusercontent.com/weaviate-tutorials/intro-workshop/main/data/jeopardy_1k.json')
data = json.loads(resp.text)  # Load data

# Parse the JSON and preview it
print(type(data), len(data))
print(json.dumps(data[1], indent=2))

env: GEMINI_API_KEY=
<class 'list'> 1000
{
  "Air Date": "2005-11-18",
  "Round": "Jeopardy!",
  "Value": 200,
  "Category": "RHYME TIME",
  "Question": "Any pigment on the wall so faded you can barely see it",
  "Answer": "faint paint"
}


In [8]:
import weaviate
from weaviate import EmbeddedOptions
import os

client = weaviate.connect_to_local(host="localhost", port=8080, grpc_port=50051, skip_init_checks=True)

# Check that Weaviate is up and live
if client.is_live():
    print("Weaviate is live!")
else:
    print("Weaviate is not reachable.")

Weaviate is live!


In [9]:
if client.collections.exists("Question"):
    client.collections.delete("Question")

In [10]:
from weaviate.classes.config import Configure, Property, DataType, VectorDistances

# Define the collection name and properties
collection_name = "Question"

# Define properties with correct field names
properties = [
    Property(name="question", data_type=DataType.TEXT),
    Property(name="answer", data_type=DataType.TEXT)
]

# Create the collection with properly configured vectorizer and vector index
client.collections.create(
    name=collection_name,
    properties=properties,
    vectorizer_config=Configure.Vectorizer.text2vec_transformers(),
    vector_index_config=Configure.VectorIndex.hnsw(
        distance_metric=VectorDistances.COSINE
    )
)


### Create an object by specifying a question, answer and category property

In [13]:
collection = client.collections.get("Question")

uuid = collection.data.insert({
    "question": "Leonardo da Vinci was born in this country.",
    "answer": "Italy",
    "category": "Culture"
})

print(uuid)
#ADD CODE HERE

73690669-9ebc-4df9-850e-28e458bd6b2b


### Read object

In [25]:
data_object = collection.query.fetch_object_by_id("73690669-9ebc-4df9-850e-28e458bd6b2b", include_vector=True)
print(data_object.properties)
print(data_object.vector)

AttributeError: 'NoneType' object has no attribute 'properties'

{'questions': None, 'answer': 'Florence, Italy', 'question': 'Leonardo da Vinci was born in this country.', 'category': 'Culture'}
{'default': [0.08583520352840424, 0.3873284161090851, 0.23866751790046692, 0.356598824262619, -0.2748798131942749, -0.11670533567667007, 0.013386230915784836, -0.4588187038898468, -0.38260379433631897, -0.23134629428386688, -0.1480213701725006, -0.9365870356559753, -0.05772349610924721, 0.00662841135635972, -0.16081734001636505, -0.23942330479621887, -0.04471248388290405, -0.27577269077301025, 0.004870975855737925, 0.2776871919631958, 0.49548429250717163, -0.11611966043710709, 0.281759113073349, 0.1717182993888855, 0.0921269953250885, -0.2345913201570511, 0.6414344310760498, 0.14396940171718597, 0.12025734782218933, 0.04874680936336517, 0.16038542985916138, 0.5639846920967102, -0.21391555666923523, -0.33206629753112793, 0.1114923283457756, 0.1090037152171135, -0.22198478877544403, -0.10111984610557556, 0.6476231813430786, 0.04632695019245148, -0.16825646162

### Update object

In [ ]:
collection.data.update(uuid=uuid, properties={"answer":"Florence, Italy"})

### Delete object

In [24]:
collection.data.delete_by_id(uuid)

data_object = collection.data.fetch_object_by_id(uuid="73690669-9ebc-4df9-850e-28e458bd6b2b", include_vector=True)

print(data_object.properties)
print(data_object.vector)

AttributeError: '_DataCollection' object has no attribute 'fetch_object_by_id'

In [ ]:
#Examine that our database is empty

json_print(client.query.aggregate("Question").with_meta_count().do())